# 1. Recurrent layers
[Recurrent Neural Network] (RNN) is a class of neural network architectures where nodes in a layers have internal connections, allowing to express temporal behaviour. There are many types of RNN layers, but they all share the same architecture. The image below shows the information flow for an observation, or for a document in the context of NLP.

<img src='image/rnn_general.png' style='height:175px; margin:20px auto;'>

Each green cell $\mathbf{x}_t\in\mathbb{R}^{V\times1}$ represents the embedding vector of a token, and each blue cell $\mathbf{h}_t\in\mathbb{R}^{D\times1}$ represents an output vector. With the input sequence size is fixed at $T$, RNN adjusts itself to match the input length. The most important part of a RNN layer is the grey cell $A$ that repeats multiple times, being account for information processing. We can see that at a time step, the output value $\mathbf{h}_t$ is influenced by all previous steps $\mathbf{h}_{t-1},\mathbf{h}_{t-2},\dots$, besides the input $\mathbf{x}_t$. This design resembles *memory* and enables RNN to capture sequential relationship.

There are many architectures for a recurrent layers, the only difference between them is how the cell $A$ being desgined. In this article, we are going to learn the cell architectures of Simple RNN, LSTM and GRU.

[Recurrent Neural Network]: https://en.wikipedia.org/wiki/Recurrent_neural_network

## 1.1. Common blocks
This section introduces common blocks in recurrent architectures. Knowing each of them separately helps us understanding compicated designs better.

### Concatenation

<img src='image/rnn_concatenation.png' style='height:100px; margin:0px auto;'>

Let's say we want to transform two input vectors $\mathbf{u}\in\mathbb{R}^{U\times1}$ and $\mathbf{v}\in\mathbb{R}^{V\times1}$ into $\mathbf{y}\in\mathbb{R}^{D\times1}$. Note that $U$ and $V$ are fixed dimensionalities of input, while $D$ is the desired output size. With weight matrices
$\mathbf{W}_{yu}\in\mathbb{R}^{D\times U},\mathbf{W}_{yv}\in\mathbb{R}^{D\times V}$
and bias vector $\mathbf{b}_y\in\mathbb{R}^{D\times1}$,
the actual formula behind the above image is:

$$\mathbf{y}=\mathbf{W}_{yu}\mathbf{u}+\mathbf{W}_{yv}\mathbf{v}+\mathbf{b}_y$$

Here, all three terms have size $(D\times1)$, same as $\mathbf{y}$. We can also view the above formula as concatenating $\mathbf{u}$ and $\mathbf{v}$ into a single input vector $\mathbf{x}\in\mathbb{R}^{(U+V)\times1}$, then scale it using a bigger weight matrix $\mathbf{W}_{yx}\in\mathbb{R}^{D\times(U+V)}$. This explains why the formula is visualized as a concatenation.

### Gate

<img src='image/rnn_gate.png' style='height:80px; margin:0px auto;'>

A gate consists of two calculation steps, (1) passing a vector into sigmoid function and (2) using it as a percentage multiplier. The sigmoid function (denoted $\sigma$) is account for producing numbers in range $(0,1)$. We can see the purpose of gates very clearly here: they control how much information should be let through.

## 1.2. Simple RNN

### Architecture
We call the vanilla architecture [Simple RNN] (1980s) to distinguish from the family name. Its cells is very simple, with only a concatenated value pass through an activation function. The activation function is usually $\tanh$ which produces values within the range $(-1,1)$, so that the network will be able to express *sentiment*. The cell architecture is described in the image and formula as follows:

<img src='image/rnn_cell.png' style='height:160px; margin:0px auto;'>

$$\mathbf{h}_t=\phi(\mathbf{W}_{hx}\mathbf{x}_t+\mathbf{W}_{hh}\mathbf{h}_{t-1}+\mathbf{b}_h)$$

A well-known issue with Simple RNN is that it only has *short-term memory*. This property is very easy to understand if you are familiar with the gradient vanishing problem of S-shaped activation functions. During [backpropagation through time] for a pair of words with large $\Delta t$, the product of partial derivatives may trigger saturation zones of $\tanh$, making the derivative of a word with respect to the other almost zero. As a result, Simple RNN fails to capture long-term memory.

[Simple RNN]: https://en.wikipedia.org/wiki/Recurrent_neural_network#Fully_recurrent
[backpropagation through time]: https://en.wikipedia.org/wiki/Backpropagation_through_time

### Sizing
$\mathbf{W}_{hx},\mathbf{W}_{hh}$ and $\mathbf{b}_h$, as explained earlier, are the weight matrices and bias vector. Their corresponding sizes are $(D\times V)$, $(D\times D)$ and $(D\times 1)$. Note that these parameters are used across cells, hence taking sum of their sizes gets us the total number of parameters need to be trained:

$$D\times(D+V+1)$$

For example, we use a BERT pretrained model to encode a corpus containing $N=10\,000$ documents. The embedding dimension is $V=512$ and documents are truncated to have $T=128$ tokens. Then a RNN layer with $D=50$ will require $50\times(50+512+1)=28\,150$ parameters to process such input.

### Implementation
RNN is implemented in TensorFlow in both layer-level and cell-level, via the classes
<code style='font-size:13px'><a href=https://www.tensorflow.org/api_docs/python/tf/keras/layers/SimpleRNN>SimpleRNN</a></code>
and
<code style='font-size:13px'><a href=https://www.tensorflow.org/api_docs/python/tf/keras/layers/SimpleRNNCell>SimpleRNNCell</a></code>.
They have the following hyperparameters:
- <code style='font-size:13px; color:firebrick'>units</code>: the dimensionality of output space ($D$).
- <code style='font-size:13px; color:firebrick'>activation</code>: the activation function ($\phi$), defaults to *tanh*.

In [1]:
from sspipe import p, px
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score as AUC

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow_hub as hub
import tensorflow_text as text

In [2]:
dfSpam = pd.read_csv('data/spam_message.csv')
xTrainRaw, xTestRaw, yTrain, yTest = train_test_split(dfSpam.content, dfSpam.spam, test_size=0.2)

In [3]:
def process_feature(corpus):
    bertProcessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
    bertEncoder = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/2')
    corpus = corpus.str.lower()
    corpus = bertProcessor(corpus)
    corpus = bertEncoder(corpus)
    corpus = corpus['sequence_output']
    return corpus
    
xTrain = process_feature(xTrainRaw)
xTest = process_feature(xTestRaw)

In [22]:
xTrain.shape

TensorShape([4457, 128, 256])

In [26]:
layers.SimpleRNN(16)(xTrain).shape

TensorShape([4457, 16])

In [27]:
layers.SimpleRNN(16, return_sequences=True)(xTrain).shape

TensorShape([4457, 128, 16])

<b style='color:navy'><i class="fa fa-info-circle"></i>&nbsp; Note</b><br>
When using RNN with other layers, there are two cases:
- The next layer being Fully Connected, then we only use the last hidden state, $\mathbf{h}_T$. The output shape in this case is  $(N\times D)$.
- The next layer being another RNN layer (including LSTM and GRU), then we need to return the full sequence $\mathbf{h}_1,\mathbf{h}_2,\dots,\mathbf{h}_T$. This is done by specifying <code style='font-size:13px'>return_sequences=True</code>. The output shape this time is $(N\times T\times D)$.

In [4]:
model = keras.Sequential([
    layers.SimpleRNN(10, return_sequences=True),
    layers.SimpleRNN(5),
    layers.Dense(1, activation='sigmoid')
])
model.compile(loss='mse', optimizer='adam')
model.fit(xTrain, yTrain)

140/140 [==============================] - 5s 25ms/step - loss: 0.0793


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128, 10)           2670      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 5)                 80        
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 2,756
Trainable params: 2,756
Non-trainable params: 0
_________________________________________________________________


In [7]:
yTestPred = model.predict(xTest)
AUC(yTest, yTestPred)

0.9789832918592251

In [9]:
_ = [print(weight.shape) for weight in model.layers[0].weights]

(256, 10)
(10, 10)
(10,)


## 1.3. LSTM

### Architecture
[LSTM] (Long Short-Term Memory) is a recurrent architecture published in 1997 that does not suffer from the gradient vanishing/exploding problem. This means, the network is able to capture long-term memory. The image below illustrates the architecture of a LSTM *memory cell*.

<img src='image/lstm_cell.png' style='height:320px; margin:0px auto;'>

The ability to manipulate information of LSTM is regulated via three gates (green node): forget $\mathbf{f}_t$, input $\mathbf{i}_t$ and output $\mathbf{o}_t$. Recall that a gate is a Sigmoid function ($\phi_1$ in this picture) followed by a multiplication. These gates give LSTM the ability to learn when to remember and when to forget, based on previous *hidden state* $\mathbf{h}_{t-1}$ and current input data $\mathbf{x}_t$. They all share the same formula, but with different parameters:

$$\begin{aligned}
\mathbf{f}_t &= \phi_1(\mathbf{W}_{fh}\mathbf{h}_{t-1}+\mathbf{W}_{fx}\mathbf{x}_t+\mathbf{b}_f) \\
\mathbf{i}_t &= \phi_1(\mathbf{W}_{ih}\mathbf{h}_{t-1}+\mathbf{W}_{ix}\mathbf{x}_t+\mathbf{b}_i) \\
\mathbf{o}_t &= \phi_1(\mathbf{W}_{oh}\mathbf{h}_{t-1}+\mathbf{W}_{ox}\mathbf{x}_t+\mathbf{b}_o) \\
\end{aligned}$$

The key component contributing to the long-term-memory ability of LSTM is the *internal state* $\mathbf{c}_t$. This variable works like a conveyor belt running straight through the entire chain, gathering *additional* information $\tilde{\mathbf{c}}_t$ at each memory cell it goes through. Because new information is really mathematically added, we are assured the gradients can pass many times without vanishing or exploding. 

$$\begin{aligned}
\tilde{\mathbf{c}}_t &= \phi_2(\mathbf{W}_{ch}\mathbf{h}_{t-1}+\mathbf{W}_{cx}\mathbf{x}_t+\mathbf{b}_c) \\
\mathbf{c}_t &= \mathbf{f}_t\odot\mathbf{c}_{t-1}+\mathbf{i}_t\odot\tilde{\mathbf{c}}_t \\
\end{aligned}$$

Finally, the current *hidden state* $\mathbf{h}_t$ is computed by $\tanh$-activating $\mathbf{c}_t$ and scaling it down by the output gate $\mathbf{o}_t$.

$$\begin{aligned}
\mathbf{h}_t &= \mathbf{o}_t\odot\phi_2(\mathbf{c}_t) \\
\end{aligned}$$

[LSTM]: https://en.wikipedia.org/wiki/Long_short-term_memory

### Steps

<img src='image/lstm_steps.png' style='height:450px; margin:0px auto;'>

Because it is quite hard to track what is going on by looking at the full architecture of LSTM, we will break it down into four steps:
- Step 1, construct the *forget gate* that decides how much old data should be kept.
- Step 2, construct the *input gate* that decides what new data should be stored.
- Step 3, compute the *internal state* by combining two processes, *forgetting* and *receiving* information.
- Step 4, construct the *output gate* and compute the output value.

### Sizing
In LSTM, all variables except the input have the same size ($D$), it is a hyperparameter set by Data Scientists. We can see in the formulation, there are 4 concatenations of $\mathbf{x}$ and $\mathbf{h}$, so that the number of parameters required to train a LSTM layer will be:

$$4\times D\times(D+V+1)$$

### Implementation
TensorFlow has both layer-level and cell-level implementations for LSTM via the classes
<code style='font-size:13px'><a href=https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM>LSTM</a></code> and
<code style='font-size:13px'><a href=https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTMCell>LSTMCell</a></code>,
with the following hyperparameters:
- <code style='font-size:13px; color:firebrick'>units</code>: the dimensionality of output space ($D$).
- <code style='font-size:13px; color:firebrick'>recurrent_activation</code>: the activation function for gates ($\phi_1$), defaults to *sigmoid*.
- <code style='font-size:13px; color:firebrick'>activation</code>: the activation function for processing data ($\phi_2$), defaults to *tanh*.

## 1.4. GRU
[Gated Recurrent Units] (GRU) was published in 2014 as an alternative that retains key idea of LSTM but is faster in computation.

[Gated Recurrent Units]: https://en.wikipedia.org/wiki/Gated_recurrent_unit

### Architecture

<img src='image/gru_cell.png' style='height:320px; margin:0px auto;'>

GRU comes with the two-gate mechanism, *reset* ($\mathbf{r}_t$) and *update* ($\mathbf{z}_t$), rather than three like LSTM. They are also based on previous *hidden state* $\mathbf{h}_{t-1}$ and current input data $\mathbf{x}_t$ like LSTM's gates.

$$\begin{aligned}
\mathbf{r}_t &= \phi_1(\mathbf{W}_{rh}\mathbf{h}_{t-1}+\mathbf{W}_{rx}\mathbf{x}_t+\mathbf{b}_r) \\
\mathbf{z}_t &= \phi_1(\mathbf{W}_{zh}\mathbf{h}_{t-1}+\mathbf{W}_{zx}\mathbf{x}_t+\mathbf{b}_z) \\
\end{aligned}$$

Next, the new information $\tilde{\mathbf{h}}_t$ is computed by two processes, *receiving* new information $\mathbf{x}_t$ and *forgetting* old information $\mathbf{h}_{t-1}$. Then, it is added to the old state weightedly (weights are controled via the update gate) to get the current hidden state $\mathbf{h}_t$. Unlike LSTM, GRU does not maintain the *internal state* but resembles its behaviour.

$$\begin{aligned}
\tilde{\mathbf{h}}_t &= \phi_2(\mathbf{W}_{hx}\mathbf{x}_t+\mathbf{W}_{hh}(\mathbf{r}_t\odot\mathbf{h}_{t-1})+\mathbf{b}_h) \\
\mathbf{h}_t &= \mathbf{z}_t\odot\mathbf{h}_{t-1}+(1-\mathbf{z}_t)\odot\tilde{\mathbf{h}}_{t-1} \\
\end{aligned}$$

The number f parameters of a GRU layer is $3\times D\times(D+V+1)$, as it has 3 concatenations of $\mathbf{x}$ and $\mathbf{h}$.

### Implementation
TensorFlow implements GRU via
<code style='font-size:13px'><a href=https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU>GRU</a></code> and
<code style='font-size:13px'><a href=https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRUCell>GRUCell</a></code>
with the same hyperparameters as LSTM.

## 1.5. Bi-directional
All the recurrent layers have been introduced so far are uni-directional, i.e. going from the left to the right. In other words, our networks only model leftward context. This design works fine for time series data, but for text data, rightward context also matters. This problem is solved by generalizing recurrent layers to [bi-directional]. The architecture is described in the following image:

<img src='image/rnn_bidirectional.png' style='height:245px; margin:20px auto;'>

We can observe that a bi-directional layer is composed of a foward sub-layer and a backward sub-layer. They can be of the same type or not, and their cells are denoted $A$ and $A'$ respectively. Their corresponding outputs $\overrightarrow{\mathbf{h}}_t$ and $\overleftarrow{\mathbf{h}}_t$ are then merged into $\mathbf{h}_t$ using various strategies, where the most common one is concatenating. Bi-directional layer is implemented in TensorFlow as a wrapper around recurrent layers via the class
<code style='font-size:13px'><a href=https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional>Bidirectional</a></code>.

[bi-directional]: https://en.wikipedia.org/wiki/Bidirectional_recurrent_neural_networks

# 2. Recurrent architectures

## 2.1. Seq2Seq
[Seq2Seq] (Sequence-to-Sequence) is a Deep Learning architecture published in 2014 mainly for [neural machine translation]. It aims to transform an input sequence to a new one, both can be of arbitrary lengths. Its design is very natural to translation problems, and has resolved the fixed-length constraint in standalone recurrent layers. The Seq2Seq uses a encoder-decoder architecture, each process uses recurrent layers such as LSTM and GRU. With $(\mathbf{x}_1,\dots,\mathbf{x}_M)$ a source sentence and $(\mathbf{y}_1,\dots,\mathbf{y}_N)$ the target sentence, the architecture of Seq2Seq is described in the following image:

The *encoder* of Seq2Seq is simply a recurrent layer where the hidden state at a time step is expressed by $(\mathbf{x}_m,\mathbf{e}_{m-1})\mapsto\mathbf{e}_m$. The last hidden state $\mathbf{e}_M$ will be used as the output, known as the *context vector* $\mathbf{c}$ (also called *sentence embedding* or *thought vector*).

The *decoder* is a bit different, in which the hidden state has the signature $(\mathbf{c},\mathbf{y}_{n-1},\mathbf{d}_{n-1})\mapsto\mathbf{d}_n$

[Seq2Seq]: https://en.wikipedia.org/wiki/Seq2seq
[neural machine translation]: https://en.wikipedia.org/wiki/Neural_machine_translation

## 2.2. Attention
[Attention] implement
<code style='font-size:13px'><a href=https://www.tensorflow.org/api_docs/python/tf/keras/layers/Attention>Attention</a></code>

[Attention]: https://en.wikipedia.org/wiki/Attention_(machine_learning)

## 2.2. Transformer
[Transformer]

[Transformer]: https://en.wikipedia.org/wiki/Transformer_(machine_learning_model)

# References
- *bioinf.jku.at - [Long Short-Term Memory](https://www.bioinf.jku.at/publications/older/2604.pdf)*
- *arxiv.org - [On the Properties of Neural Machine Translation: Encoder–Decoder Approaches](https://arxiv.org/pdf/1409.1259.pdf)*
- *arxiv.org - [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/pdf/1409.3215v3.pdf)*
- *arxiv.org - [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/pdf/1409.0473.pdf)*
- *arxiv.org - [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/pdf/1508.04025.pdf)*
- *arxiv.org - [Attention is All You Need](https://arxiv.org/pdf/1706.03762.pdf)*
- *amitness.com - [Recurrent Keras layer](https://amitness.com/2020/04/recurrent-layers-keras/)*
- *colah.github.io - [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)*
- *d2l.ai - [Recurrent Neural Networks](https://d2l.ai/chapter_recurrent-neural-networks/rnn.html)*
- *d2l.ai - [Long Short-Term Memory](https://d2l.ai/chapter_recurrent-modern/lstm.html)*
- *d2l.ai - [Gated Recurrent Units](https://d2l.ai/chapter_recurrent-modern/gru.html)*
- *distill.pub - [Memorization in RNNs](https://distill.pub/2019/memorization-in-rnns/)*
- *distill.pub - [Augumented RNNs](https://distill.pub/2016/augmented-rnns/)*
- *lilianweng.github.io - [Attention? Attention!](https://lilianweng.github.io/posts/2018-06-24-attention/)*
- *blog.floydhub.com - [Attention mechanism](https://blog.floydhub.com/attention-mechanism/amp/)*
---
- https://www.kaggle.com/code/tanulsingh077/deep-learning-for-nlp-zero-to-transformers-bert
- https://www.kaggle.com/code/kredy10/simple-lstm-for-text-classification